In [1]:
import pandas as pd

In [2]:
# 사진 제외 : 87%
# 사진 포함 : 88%

In [3]:
df_naver = pd.read_csv('A_V_Adj_C_df_naver_post_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep = pd.read_csv('A_V_Adj_C_df_dbdbdeep_post_table.csv',sep='\t',encoding='utf-8')
df_naver_page = pd.read_csv('A_V_Adj_C_df_naver_page_table.csv',sep='\t',encoding='utf-8')
df_dbdbdeep_page = pd.read_csv('A_V_Adj_C_df_dbdbdeep_page_table.csv',sep='\t',encoding='utf-8')

In [4]:
df_dbdbdeep.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       514
건강·의학        917
공연·전시        378
교육·학문        732
국내여행          80
드라마·방송       383
등산·낚시·레저      39
만화·애니         30
맛집           662
사진            31
스포츠           64
시사·인문·경제      95
어학·외국어       179
와인·술          68
육아·결혼         55
자동차          330
차·커피·디저트     569
패션·뷰티       1238
Name: label, dtype: int64

In [5]:
df_naver.groupby(by='Category')['label'].count()

Category
IT·컴퓨터       476
건강·의학        565
공연·전시        532
교육·학문        492
국내여행         475
드라마·방송       258
등산·낚시·레저     232
만화·애니        653
맛집          1033
사진          1682
스포츠          841
시사·인문·경제     565
어학·외국어       162
와인·술          80
육아·결혼        257
자동차          643
차·커피·디저트     254
패션·뷰티        742
Name: label, dtype: int64

In [6]:
df_naver = df_naver.dropna(subset=['Embedding_text'])
df_dbdbdeep = df_dbdbdeep.dropna(subset=['Embedding_text'])

In [7]:
def Embedding_text_reduction(Embedding_text):
    if len(Embedding_text.split(' '))> 55:
        return " ".join(Embedding_text.split(' ')[:-28])
    elif len(Embedding_text.split(' '))>45:
        return " ".join(Embedding_text.split(' ')[:-12])
    elif len(Embedding_text.split(' '))>35:
        return " ".join(Embedding_text.split(' ')[:-5])
    else:
        return " ".join(Embedding_text.split(' '))

In [8]:
df_naver['Embedding_text'] = df_naver['Embedding_text'].apply(lambda x : Embedding_text_reduction(x))

In [9]:
# df_naver = df_naver[df_naver['Category'] != '사진']
# df_dbdbdeep = df_dbdbdeep[df_dbdbdeep['Category'] != '사진']

In [10]:
from sklearn.utils import shuffle
df_naver = shuffle(df_naver)

In [11]:
#df_dbdbdeep.shape

In [12]:
#df_naver = df_naver.iloc[:6178]

In [13]:
Total = pd.concat([df_naver,df_dbdbdeep,df_naver_page,df_dbdbdeep_page])

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [14]:
Total = Total.dropna(subset=['Embedding_text'])

In [15]:
Text = Total['Embedding_text'].tolist()

## TFIDF

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(max_features=100)
x = v.fit_transform(Text)

### TFIDF model save

In [17]:
import pickle
import numpy as np
#Save vectorizer.vocabulary_
pickle.dump(v.vocabulary_,open("Text_features_100.pkl","wb"))

# Model_load

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer()

In [19]:
loaded_vec = TfidfVectorizer(decode_error="replace",vocabulary=pickle.load(open("Text_features_100.pkl", "rb")))

In [20]:
x = Total['Embedding_text'].apply(lambda x : transformer.fit_transform(loaded_vec.fit_transform(np.array([x]))))

In [21]:
Array = list(map(lambda x : x.toarray(),x))

In [22]:
Label = Total['label']

In [23]:
test_df = pd.DataFrame({'Array':Array,'label':Label})

In [24]:
test_df = test_df.dropna()

In [25]:
from sklearn.utils import shuffle
test_df = shuffle(test_df)

In [26]:
x = test_df['Array'].apply(lambda x: x.flatten())
x = x.tolist()

In [27]:
y = test_df['label'].tolist()

In [28]:
# from sklearn.utils import shuffle
# test = shuffle(test)
# test = test.sample(frac=1)

## Model_Experiment

In [29]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x , y , test_size=0.33, random_state=42)

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

result = LinearSVC(random_state=0).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.8617085427135679


In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result.predict(x_test)))

             precision    recall  f1-score   support

        0.0       0.84      0.81      0.83      2036
        1.0       0.87      0.90      0.88      2939

avg / total       0.86      0.86      0.86      4975



In [32]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(result, x, y, cv=5)

scores                                              

array([0.86898839, 0.87396352, 0.86401327, 0.86197744, 0.8715992 ])

## Gausian NB

In [33]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(x_train, y_train)
clf.predict(x_test)
print("NB :",accuracy_score(clf.predict(x_test),y_test))

NB : 0.820502512562814


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

        0.0       0.75      0.83      0.79      2036
        1.0       0.87      0.81      0.84      2939

avg / total       0.83      0.82      0.82      4975



In [35]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.82354892, 0.82321725, 0.81890547, 0.81353683, 0.83244857])

## Randomforest

In [36]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(x_train, y_train)

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [37]:

print("RF :",accuracy_score(clf.predict(x_test),y_test))

RF : 0.7762814070351759


In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

        0.0       0.87      0.54      0.66      2036
        1.0       0.75      0.94      0.83      2939

avg / total       0.80      0.78      0.76      4975



In [39]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.77910448, 0.79104478, 0.77147595, 0.78201725, 0.79595222])

## Logsitic Regression

In [40]:
from  sklearn  import  linear_model
logreg = linear_model.LogisticRegression(C=2.0,random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
logreg.fit(x_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=42, solver='sag',
          tol=0.0001, verbose=0, warm_start=True)

In [41]:
print("RF :",accuracy_score(logreg.predict(x_test),y_test))

RF : 0.862713567839196


In [42]:
from sklearn.metrics import classification_report
print(classification_report(y_test, logreg.predict(x_test)))

             precision    recall  f1-score   support

        0.0       0.85      0.81      0.83      2036
        1.0       0.87      0.90      0.89      2939

avg / total       0.86      0.86      0.86      4975



In [43]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(logreg, x, y, cv=5)
scores                                              

array([0.86865672, 0.87495854, 0.86368159, 0.86297279, 0.86960849])

## Decision Tree 

In [44]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf = clf.fit(x_train, y_train)

In [45]:
print("DT :",accuracy_score(clf.predict(x_test),y_test))


DT : 0.7841206030150754


In [46]:
from sklearn.metrics import classification_report
print(classification_report(y_test, clf.predict(x_test)))

             precision    recall  f1-score   support

        0.0       0.74      0.73      0.73      2036
        1.0       0.81      0.82      0.82      2939

avg / total       0.78      0.78      0.78      4975



In [47]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, x, y, cv=5)
scores                                              

array([0.79004975, 0.79402985, 0.77313433, 0.7743862 , 0.79727936])

## MLP

In [48]:
import time
from sklearn.neural_network import MLPClassifier

In [49]:
mlp_clf = MLPClassifier(
    hidden_layer_sizes=(50,),
    max_iter=35,
    alpha=1e-4,
    solver='sgd',
    verbose=10,
    tol=1e-4,
    random_state=1,
    learning_rate_init=.1
)

In [50]:
start = time.time()
mlp_clf.fit(x_train, y_train)
end = time.time()
print('Time: {:f}s'.format(end-start))

Iteration 1, loss = 0.57983937
Iteration 2, loss = 0.35908439
Iteration 3, loss = 0.32573120
Iteration 4, loss = 0.32007679
Iteration 5, loss = 0.31807734
Iteration 6, loss = 0.31715078
Iteration 7, loss = 0.31592271
Iteration 8, loss = 0.31582633
Iteration 9, loss = 0.31383355
Iteration 10, loss = 0.31384717
Iteration 11, loss = 0.31238460
Iteration 12, loss = 0.31146634
Iteration 13, loss = 0.30877022
Iteration 14, loss = 0.30842177
Iteration 15, loss = 0.30693345
Iteration 16, loss = 0.30514031
Iteration 17, loss = 0.30302447
Iteration 18, loss = 0.30198294
Iteration 19, loss = 0.30031109
Iteration 20, loss = 0.29770017
Iteration 21, loss = 0.29748683
Iteration 22, loss = 0.29451736
Iteration 23, loss = 0.29369916
Iteration 24, loss = 0.29059317
Iteration 25, loss = 0.28980086
Iteration 26, loss = 0.28730343
Iteration 27, loss = 0.28509838
Iteration 28, loss = 0.28271123
Iteration 29, loss = 0.28011185
Iteration 30, loss = 0.27855619
Iteration 31, loss = 0.27615241
Iteration 32, los

C:\Users\AjouHCI\Anaconda3\envs\py\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (35) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [51]:
y_pred = mlp_clf.predict(x_test)
print("테스트 정확도: {:.3f}".format(accuracy_score(y_pred, y_test)))

테스트 정확도: 0.873


In [52]:
# import tensorflow as tf
# import random
# import numpy as np

In [53]:
# tf.reset_default_graph()

In [54]:
# # hyper Parameter
# learning_rate = 0.001
# training_epochs = 15
# batch_size = 100

In [55]:
# # input layer
# X = tf.placeholder(tf.float32, [None, 134])
# Y = tf.placeholder(tf.float32, [None, 1])

# # dropout
# keep_prob = tf.placeholder(tf.float32)

# # Xavier_Initializer
# xavier_init = tf.contrib.layers.xavier_initializer()

In [56]:
# # Hidden layers and Output layer
# W1 = tf.get_variable("W1", shape=[134, 67], initializer=xavier_init)
# b1 = tf.Variable(tf.random_normal([67]))
# L1 = tf.nn.relu(tf.matmul(X, W1)+b1)
# dropout1 = tf.nn.dropout(L1, keep_prob=keep_prob)

# W2 = tf.get_variable("W2", shape=[67, 30], initializer=xavier_init)
# b2 = tf.Variable(tf.random_normal([30]))
# L2 = tf.nn.relu(tf.matmul(dropout1, W2)+b2)
# dropout2 = tf.nn.dropout(L2, keep_prob=keep_prob)

# W3 = tf.get_variable("W3", shape=[30, 1], initializer=xavier_init)
# b3 = tf.Variable(tf.random_normal([1]))
# hypothesis = tf.matmul(dropout2, W3)+b3

In [57]:
# # define cost/loss & optimizer
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
# # Initialize
# sess = tf.Session()
# sess.run(tf.global_variables_initializer())

In [58]:
# x_train = np.array(x_train)
# x_test = np.array(x_test)
# y_train = np.array(y_train)
# y_test = np.array(y_test)

In [59]:
# x_train[0].reshape(1,134)

In [60]:
# x_train[1].shape

In [61]:
# # Train Model
# for epoch in range(training_epochs):
#     feed_dict = {X:x_train, Y: y_train, keep_prob:0.7}
#     c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
#     avg_cost += c / total_batch

#     print('Epoch:', '{:04d}'.format(epoch +1), 'cost =', '{:.9f}'.format(avg_cost))

# print('Training Finished')

In [62]:
# text_x = [0,0,1,0]
# np.transpose(text_x)